In [1]:
import pyodbc
import pandas as pd 
import requests
import geocoder
from geopy.geocoders import Nominatim
from IPython.display import clear_output
from pprint import pprint
import time
import re
import timeit
from IPython.display import clear_output

In [182]:
cnxn = pyodbc.connect('DSN=SQLAmlin;UID=MSAmlin;PWD=password')

In [3]:
cur = cnxn.cursor()

### Cambridgeshire

In [6]:
cambs_query = 'select * from [initial].[Cambridgeshire]'

In [85]:
cambs_df = pd.read_sql(cambs_query, cnxn)

In [86]:
def cambs_location_clean(cambs_df):
    # get rid of weird characters
    cambs_df['title'] = cambs_df['title'].map(lambda x: x.replace('â€“', '-'))
    # incident type stored with address - split on ' - ' to separate, then drop the first list item, then join to connect elements of address
    cambs_df['LookupAddress'] = cambs_df['title'].map(lambda x: ', '.join(x.split(' - ')[1:]))
    # when locations have only 1 word, put cambridgeshire in the address to increase chance of OSM finding the right one
    cambs_df['LookupAddress'] = cambs_df['LookupAddress'].map(lambda x: x + ', Cambridgeshire' if (x.count(', ') == 0)  else x)
    return cambs_df

### Cornwall

In [133]:
corn_query = 'select * from initial.Cornwall'
corn_df = pd.read_sql(corn_query, cnxn)

In [132]:
def corn_location_clean(corn_df):
    corn_df['LookupAddress'] = corn_df['location'].fillna('')
    corn_df['LookupAddress'] = corn_df['LookupAddress'].map(lambda x: x.replace('near', ''))
    return corn_df

### Cumbria

In [120]:
cumb_query = 'select * from initial.Cumbria'
cumb_df = pd.read_sql(cumb_query, cnxn)

In [135]:
def cumb_location_clean(cumb_df):
    cumb_df['LookupAddress'] = cumb_df['location'].map(lambda x: x[1:-1] + ', Cumbria')
    return cumb_df

### Devon

In [121]:
dev_query = 'select * from [initial].[Devon]'
dev_df = pd.read_sql(dev_query, cnxn)

In [138]:
def dev_location_clean(dev_df):
    dev_df['LookupAddress'] = dev_df['Location']
    return dev_df

### Essex

In [122]:
essex_query = 'select * from [initial].[Essex]'
essex_df = pd.read_sql(essex_query, cnxn)

In [142]:
def essex_location_clean(essex_df):
    essex_df['LookupAddress'] = essex_df['Location']
    return essex_df

### Hampshire

In [123]:
hamp_query = 'select * from [initial].[Hampshire]'
hamp_df = pd.read_sql(hamp_query, cnxn)

In [144]:
def hamp_location_clean(hamp_df):
    hamp_df['LookupAddress'] = hamp_df['Address']
    hamp_df['LookupAddress'] = hamp_df['LookupAddress'].fillna('None')
    hamp_df['LookupAddress'] = hamp_df['LookupAddress'].map(lambda x: x.replace('<div class="addressDisplay">', '').replace('</div>', '').replace('---', ' ').strip())
    return hamp_df

### Kent

In [124]:
kent_query = 'select * from [initial].[Kent]'
kent_df = pd.read_sql(kent_query, cnxn)

In [151]:
def kent_location_clean(kent_df):
    kent_df['LookupAddress'] = kent_df['Map - location shows general area of incident']
    return kent_df

### Lancashire

In [125]:
lancs_query = 'select * from [initial].[Lancashire]'
lancs_df = pd.read_sql(lancs_query, cnxn)

In [154]:
def lancs_location_clean(lancs_df):
    lancs_df['LookupAddress'] = lancs_df['Address']
    return lancs_df

### Leicestershire

In [159]:
leics_query = 'select * from [initial].[Leicestershire]'
leics_df = pd.read_sql(leics_query, cnxn)

In [160]:
def leics_location_clean(leics_df):
    leics_df['LookupAddress'] = leics_df['Incident Location:'].map(lambda x: x.replace('Incident Location:', ''))
    return leics_df

### Nottinghamshire

In [127]:
notts_query = 'select * from [initial].[Nottinghamshire]'
notts_df = pd.read_sql(notts_query, cnxn)

In [163]:
def notts_location_clean(notts_df):
    notts_df['LookupAddress'] = notts_df['Incident Location']
    return notts_df

### Scotland

In [128]:
scot_query = 'select * from [initial].[Scotland]'
scot_df = pd.read_sql(scot_query, cnxn)

In [109]:
def scot_location_clean(scot_df):
    scot_df['LookupAddress'] = scot_df['DZ11NM'].map(lambda x: re.search('[A-Za-z ]+', x).group()) + ', ' + scot_df['CouncilArea']
    return scot_df

### Shropshire

In [176]:
shrop_query = 'select * from [initial].[Shropshire]'
shrop_df = pd.read_sql(shrop_query, cnxn)

In [173]:
def shrop_location_clean(shrop_df):
    shrop_df['LookupAddress'] = shrop_df['location']
    return shrop_df

### Surrey

In [130]:
surrey_query = 'select * from [initial].[Surrey]'
surrey_df = pd.read_sql(surrey_query, cnxn)

In [114]:
def surrey_location_clean(surrey_df):
    surrey_df['LookupAddress'] = surrey_df['Ward Lable'] + ', ' + surrey_df['District lable']
    return surrey_df

### West Yorkshire

In [131]:
wy_query = 'select * from [initial].[WestYorkshire]'
wy_df = pd.read_sql(wy_query, cnxn)

In [179]:
def wy_location_clean(wy_df):
    wy_df['LookupAddress'] = wy_df['Location']
    return wy_df

### Avon, Berkshire, Staffordshire and West Mids

In [194]:
# These have location information but at too coarse a level of granularity

avon_query = 'select * from [initial].[Avon]'
berks_query = 'select * from [initial].[RoyalBerkshire]'
staffs_query = 'select * from [initial].[Staffordshire]'
wm_query = 'select * from [initial].[WestMidlands]'
lon_arch_query = 'select * from [initial].[LondonArchived]'
avon_df = pd.read_sql(avon_query, cnxn)
berks_df = pd.read_sql(berks_query, cnxn)
staffs_df = pd.read_sql(staffs_query, cnxn)
wm_df = pd.read_sql(wm_query, cnxn)
lon_arch_df = pd.read_sql(lon_arch_query, cnxn)

In [195]:
def avon_location_clean(avon_df):
    avon_df['LookupAddress'] = 'UNKNOWN UNKNOWN'
    return avon_df

def berks_location_clean(berks_df):
    berks_df['LookupAddress'] = 'UNKNOWN UNKNOWN'
    return berks_df

def staffs_location_clean(staffs_df):
    staffs_df['LookupAddress'] = 'UNKNOWN UNKNOWN'
    return staffs_df

def wm_location_clean(wm_df):
    wm_df['LookupAddress'] = 'UNKNOWN UNKNOWN'
    return wm_df

def lon_arch_location_clean(lon_arch_df):
    lon_arch_df['LookupAddress'] = 'UNKNOWN UNKNOWN'
    return lon_arch_df